In [4]:
import numpy as np

train_data = np.load("C:/Users/sdat7/home/2021/기계학습/[2021 Spring Class] Ajou ML by JBR/train_data.npy", allow_pickle=True)
train_label = np.load("C:/Users/sdat7/home/2021/기계학습/[2021 Spring Class] Ajou ML by JBR/train_label.npy", allow_pickle=True)

train_data.shape

(2682,)

In [5]:
for i in train_data:
    print(len(i))

265
245
246
294
29
66
169
50
50
81
108
136
207
33
93
29
95
68
315
132
138
44
370
244
89
284
125
383
327
302
48
62
166
254
318
110
86
196
203
25
59
51
250
48
160
154
29
17
140
61
403
90
209
74
21
214
191
28
89
207
68
315
49
107
39
162
265
140
39
105
158
76
67
42
46
294
89
196
143
269
191
19
189
27
202
254
141
240
39
207
361
73
214
214
26
80
35
224
78
85
109
49
30
23
81
227
352
34
60
91
398
141
141
319
189
244
267
190
161
282
207
232
191
69
231
208
211
195
54
186
215
254
34
209
109
236
299
272
216
225
160
243
174
298
122
189
220
268
161
250
262
245
276
182
280
203
218
166
166
221
197
178
258
287
318
116
225
226
165
299
118
290
270
180
205
175
208
243
234
73
235
178
163
199
122
189
224
264
215
208
218
195
270
234
254
308
160
207
258
156
208
197
244
252
150
200
258
251
118
162
162
170
313
215
222
149
117
34
222
134
119
54
115
175
27
154
283
117
194
297
205
157
205
298
141
275
60
103
35
227
53
282
258
286
220
206
254
272
168
104
135
296
274
188
206
83
176
295
109
231
114
164
179
158
226
186